[Google Vision python docs](https://googleapis.dev/python/vision/latest/index.html)

cf. [another tutorial](https://cloud.google.com/functions/docs/tutorials/ocr), which uses cloud functions

It turns out, google OCR will work with PDFs only if thy are stored in google cloud. Since that just adds another layer of complexity, I'm going to work with Backblaze

In [5]:
from PIL import ImageDraw
from google.oauth2 import service_account

from oed.bounding_boxes import display_annotation

creds_path = '/home/src/oed/creds.json'

creds = service_account.Credentials.from_service_account_file(creds_path)

In [2]:
%connect_info

{
  "shell_port": 42003,
  "iopub_port": 54725,
  "stdin_port": 55791,
  "control_port": 33279,
  "hb_port": 36189,
  "ip": "127.0.0.1",
  "key": "300b0949-9afb36e8a5d80d53cbddaa13",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-f415e2b6-0728-47c4-adec-437101ba2fbd.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [3]:
from google.protobuf.json_format import MessageToDict
from google.cloud import vision
client = vision.ImageAnnotatorClient(credentials=creds)


feature = vision.Feature(
    type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)
# async_request = vision.AsyncAnnotateFileRequest(
#     features=[feature], input_config=input_config,
#     output_config=output_config)

source_uri = 'https://jeroen.github.io/images/testocr.png'
# source_uri = 'https://africau.edu/images/default/sample.pdf'

source_uri = 'https://ohuiginn.net/tmp/singlepage-1.png'
image_src = vision.ImageSource(image_uri = source_uri)
img = vision.Image(source=image_src)
page_as_protobuf = client.annotate_image({
    'image': img,

# 'image': {'source': {'image_uri': source_uri}},
    'features': [feature]
})
page_as_protobuf
page_as_dict = MessageToDict(page_as_protobuf._pb)

In [6]:
# cargo-cult from https://towardsdatascience.com/how-to-extract-the-text-from-pdfs-using-python-and-the-google-cloud-vision-api-7a0a798adc13


display_annotation()

NameError: name 'creds' is not defined

coordinates are (0,0) in top right